In [1]:
import sys, os
import numpy as np

sys.path.insert(0, "/Users/Shared/auto/07p/python")
from auto import *
import scipy.optimize as op

In [2]:
Kf=10; Kb=0.4; Kp=0.2; Kc=0.2; Kh=0.08; tauP=1;
Vs=0.9; Kbar=1.957e-5; Ks=0.2;
gamma=5.5; ct=0.1; h=0.001;
Kplc=0.1; R_act=0.51;
    
def f(x):
    
    m = x[0]**4/(x[0]**4+Kc**4)
    h_inf = Kh**4/(Kh**4+x[0]**4)
    b = x[1]**2/(Kp**2+x[1]**2)
    a = 1-b
    beta = b*m*h
    alpha = a*(1-m*h_inf)
    Po = beta/(beta+Kb*(beta+alpha))
    PLC = R_act*x[0]**2/(Kplc**2+x[0]**2)
    ce = gamma*(ct-x[0])
    Jserca = Vs*(x[0]**2-Kbar*ce**2)/(x[0]**2+Ks**2)
    Jipr = Kf*Po*(ce-x[0])
    
    return np.array([Jipr-Jserca,tauP*(PLC-x[1])])

sol = op.root(f,x0=np.array([6.07518337580197e-7,1.882351936069223e-10]))
sol.x

array([0.0023753 , 0.00028758])

In [3]:
sol

    fjac: array([[-0.41212677,  0.91112652],
       [-0.91112652, -0.41212677]])
     fun: array([-1.14624820e-18,  2.71050543e-18])
 message: 'The solution converged.'
    nfev: 26
     qtf: array([ 4.00464563e-12, -3.40783127e-14])
       r: array([ 0.26637965, -0.77496047,  0.4109331 ])
  status: 1
 success: True
       x: array([0.0023753 , 0.00028758])

In [4]:
f(sol.x)

array([-1.14624820e-18,  2.71050543e-18])

In [5]:
import sys
print(sys.version)

2.7.14 |Anaconda custom (64-bit)| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [6]:
cl

<function AUTOCommands.cl>

In [7]:
Eq = run(e='hep_fixed_h',c='hep_fixed_h.eq',DS='-')

Starting hep_fixed_h ...

  BR    PT  TY  LAB       ct         L2-NORM          h          PAR(5)        PAR(6)        PAR(7)        PAR(8)        PAR(80)    
   1     1  EP    1   1.00000E-01   2.39265E-03   1.00000E-03   2.37530E-03   2.87580E-04   2.37530E-03   2.87580E-04   2.00000E+00
   1    26  BP    2   1.69501E-09   7.69453E-12   1.00000E-03  -7.69453E-12  -4.18393E-20  -7.69453E-12  -4.18393E-20   2.00000E+00
   1    27  EP    3  -9.99718E-03   2.37480E-04   1.00000E-03  -2.37463E-04   2.87580E-06  -2.37463E-04   2.87580E-06   1.00000E+00

  BR    PT  TY  LAB       ct         L2-NORM          h          PAR(5)        PAR(6)        PAR(7)        PAR(8)        PAR(80)    
   2     2  EP    4  -2.05427E-05   5.12285E-07   1.00000E-03   5.12285E-07   1.33843E-11   5.12285E-07   1.33843E-11   2.00000E+00

  BR    PT  TY  LAB       ct         L2-NORM          h          PAR(5)        PAR(6)        PAR(7)        PAR(8)        PAR(80)    
   2   200        5   1.75281E+00   9.29257E-

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def auto_to_csv(branch,name,solution_type):
    """ The goal of this function is to save as a csv file the results of an AUTO continuation 
        and return a panda dataframe of the data"""
    """ Input:
        branch: an AUTO branch solution acquired using run(blablabla)
        name: The name of the output csv file, must be a string
        solution_type: 'EQ' for equilibrium, 'PR' for periodic orbit, 'HB' for Hopf two parameters continuation
            'SNP' Saddle node of periodic orbits
        
        Output:
        panda dataframe of the branch solution"""
    
    
    """Relabelling the branch solution """
    branch = rl(branch)
    """Saving the branch solution"""
    sv(branch,name)
    
    """Acquire the b. file"""
    name_b = 'b.' + name
    content = None
    with open(name_b, 'r') as f:
        content = f.readlines()

    """Read the file from the beginning of the interesting part (depends on its type)"""
    if solution_type == 'EQ':
        content_csv = [[el for el in content[14].split(' ') if len(el) > 0 and el != '\n']]
        
    elif solution_type == 'PR':
        content_csv = [[el for el in content[16].split(' ') if len(el) > 0 and el != '\n']]
        
    elif solution_type == 'HB':
        content_csv = [[el for el in content[17].split(' ') if len(el) > 0 and el != '\n']]
    
    elif solution_type == 'SNP':
        content_csv = [[el for el in content[18].split(' ') if len(el) > 0 and el != '\n']]
        
    elif solution_type == 'SN':
        content_csv = [[el for el in content[17].split(' ') if len(el) > 0 and el != '\n']]
    else:
        print('Solution type not supported')
        return;
    
        
    """ Rename the branch"""
    content_csv[0][0] = 'branch'
    column_names = content_csv[0]
    
    """Split and read the content"""
    for line in content:
        dummy = line.split(' ')
        dummy = [el for el in dummy if len(el) > 0 and el != '\n']
        if dummy[0] == '0':
            continue

        for el_i, el in enumerate(dummy):
            if el_i < 4:
                dummy[el_i] = int(el)
            else:
                dummy[el_i] = float(el)

        if len(dummy) > 1:
            content_csv.append(dummy)
        
    # transform into panda data frame
    print(column_names)
    df = pd.DataFrame(content_csv, columns=column_names)
    
    # Export to csv
    name_csv = name + '.csv'
    df[1:-1].to_csv(name_csv)

        
    
    
    
    return df[1:-1]

In [ ]:
df_eq = auto_to_csv(Eq,'Eq','EQ')

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
trace_eq = Scatter(x=df_eq['ct'],
                  y=df_eq['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
data = [trace_eq]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='[Ct]'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_eqs = []

for h in np.linspace(0.0031,0.41,500):
    initial = run(e='hep_fixed_h',c='hep_fixed_h.eq',RL1 = h,ICP = [2])
    Eq = run(initial('EP2'),c='hep_fixed_h.eq',ICP =  [1,2,5,6,7,8,80])
   
    df_eqs.append(auto_to_csv(Eq,'Eq_h'+str(h),'EQ'))
    